In [1]:
import numpy as np
import pandas as pd
import math
import os
import re
import heapq
from IPython.display import display

In [2]:
# 打开文件
f = open('HW4_1.txt')
# 读取文章，并删除每行结尾的换行符
docs = pd.Series(f.read().splitlines())

# 打开文件
f = open('HW4_1.txt')
# 读取文章，并删除每行结尾的换行符
sentences = pd.Series(f.read().splitlines())

# 转换为小写，并使用正则表达式进行切割
docs = docs.apply(lambda x: re.split('[^a-zA-Z-]', x.lower()))
# 删除空串
for line in docs:
    while '' in line:
        line.remove('')
print(docs)

0     [vae, variational, autoencoder, for, heterogen...
1     [fr-fmss, federated, recommendation, via, fake...
2     [transfer, learning, in, collaborative, recomm...
3     [mitigating, confounding, bias, in, recommenda...
4     [fedrec, lossless, federated, recommendation, ...
                            ...                        
75    [fedrec, federated, recommendation, with, expl...
76    [cofi-points, collaborative, filtering, via, p...
77    [transfer, learning, from, app, domain, to, ne...
78    [toccf, time-aware, one-class, collaborative, ...
79    [rbpr, role-based, bayesian, personalized, ran...
Length: 80, dtype: object


In [5]:
# 统计文档集字典，加入set去重
term_set = set()
for line in docs:
    term_set.update(set(line))

# 统计df，即每个词项在文档集中的出现次数（每个文档中只记一次）
idf = [0 for i in range(len(term_set))]
for i, word in enumerate(term_set):
    df = list(word in line for line in docs).count(True)
    idf[i] = math.log(len(docs) / df, 10)
    
# 计算tf-idf
tf_idf = [[] for i in range(len(docs))]
for i in range(len(docs)):
    index, doc = i, docs[i]
    
    # 统计每个词项在每条文档中的出现次数。
    tf = [0 if doc.count(term) == 0 else 1 + math.log(doc.count(term), 10) for term in term_set]
    
    # 计算tf-idf
    tf_idf[i] = np.multiply(tf, idf)
    print(tf_idf[i])


[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.67264107
 0.         0.32330639 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.45593196 0. 

KeyboardInterrupt: 

In [4]:
# 计算文档间两两的余弦相似度
cosine_similaritys = [[] for i in range(len(docs))]
for i in range(len(docs)):
    for j in range(len(docs)):
        index1, index2 = i+1, j+1
        doc1, doc2 = sentences[i], sentences[j]

        # 取出两个文档的tf-idf向量
        v1 = tf_idf[i]
        v2 = tf_idf[j]

        # 计算余弦相似度
        cos = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

        cosine_similaritys[i].append([cos, index1, index2, doc1, doc2])

for cosine_similarity in cosine_similaritys:
    cosine_similarity.sort(key=lambda x: x[0], reverse=True)

for cosine_similarity in cosine_similaritys[:10]:
    print('====================================================================================================')
    for i in range(1, 4):
        print('---------------------------------------------------------------------------------------------------------------------------------------')
        print('第{}相似:'.format(i))
        print('文档{}和文档{}的相似度为{}'.format(cosine_similarity[i][1], cosine_similarity[i][2], round(cosine_similarity[i][0], 2)))
        print('文档{}的内容为{}'.format(cosine_similarity[i][1], cosine_similarity[i][3]))
        print('文档{}的内容为{}'.format(cosine_similarity[i][2], cosine_similarity[i][4]))


---------------------------------------------------------------------------------------------------------------------------------------
第1相似:
文档1和文档74的相似度为0.57
文档1的内容为VAE++: Variational AutoEncoder for Heterogeneous One-Class Collaborative Filtering
文档74的内容为Staged Variational AutoEncoder for Heterogeneous One-Class Collaborative
---------------------------------------------------------------------------------------------------------------------------------------
第2相似:
文档1和文档60的相似度为0.3
文档1的内容为VAE++: Variational AutoEncoder for Heterogeneous One-Class Collaborative Filtering
文档60的内容为Transfer Learning for Heterogeneous One-Class Collaborative Filtering
---------------------------------------------------------------------------------------------------------------------------------------
第3相似:
文档1和文档73的相似度为0.26
文档1的内容为VAE++: Variational AutoEncoder for Heterogeneous One-Class Collaborative Filtering
文档73的内容为Tri-task Variational Autoencoder for Modeling of Biased and Unbiased Unary Feedback 